<a href="https://colab.research.google.com/github/Shaandili/robotics-me639-2022/blob/Shaandili/ME_639_Assignment_3_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#inputs = no. of links, dh parameters, revolute or prismatic?
#output = jacobian
import math as m
import numpy as np
links = int(input()) #no. of links (if spherical - include as 3 revolute joints)
dh = np.zeros((links,4)) #matrix with denavit hartenburg parameters
print("enter dh parameters for each link in the order d,theta,r,alpha")
for i in range(links-1):
  a = list(map(float,input().strip().split()))[:4]
  dh[i] = a
#calculation of transformation matrices:
H = [0]*(links) #array with link to link transformation matrices
H[0] = np.identity(4)
H0 = np.identity(4) #overall transformation matrix
for i in range(links-1):
  d = dh[i][0]
  theta = dh[i][1]
  r = dh[i][2]
  alpha = dh[i][3]
  Z = [[m.cos(theta), -1*m.sin(theta), 0, 0], #transforation matrix, Z operations
       [m.sin(theta), m.cos(theta), 0, 0],
       [0, 0, 1, d],
       [0, 0, 0, 1]]
  X = [[1, 0, 0, r], #transforation matrix, X operations
       [0, m.cos(alpha), -1*m.sin(alpha), 0],
       [0, m.sin(alpha), m.cos(alpha), 0],
       [0, 0, 0, 1]]
  H[i+1] = np.matmul(Z,X) #adding current link to link transformation to H matrix
  H0 = H0@H[i+1] #mutliplying link to link transformations to get overall transformation

In [ ]:
on0 = [H0[0][3],H0[1][3],H0[2][3]] #position of last origin (end effector) wrt base
o = np.zeros(3) #for storing position of end effector wrt current frame
J = np.zeros((6,links)) #jacobian (initialized)
rorp = input("linkwise, enter type of joint (include end effector too): ").strip().split() #revolute or prismatic joint?

In [ ]:
h = np.identity(4) #matrix representing transformation from base to current frame
for i in range(links):
  h = h@H[i]
  z = [h[0][2], h[1][2], h[2][2]] #z -> last column of rotation
  if rorp[i] == "prismatic": #assigning of values to jacobian column in case of prismatic joint
    J[0][i] = z[0]
    J[1][i] = z[1]
    J[2][i] = z[2]
    J[3][i] = 0
    J[4][i] = 0
    J[5][i] = 0
  else: #assigning of values to jacobian column in case of revolute joint
    oi0 = [h[0][3], h[1][3], h[2][3]] #position of current frame origin wrt base origin
    #calculation of o:
    o[0] = on0[0] - oi0[0] 
    o[1] = on0[1] - oi0[1]
    o[2] = on0[2] - oi0[2]
    zxoi = (z[1]*o[2]) - (o[1]*z[2]) 
    zxoj = (o[0]*z[2]) - (z[0]*o[2])
    zxok = (z[0]*o[1]) - (o[0]*z[1])
    zxo = [zxoi, zxoj, zxok] #cross product of z and o
    J[0][i] = zxo[0]
    J[1][i] = zxo[1]
    J[2][i] = zxo[2]
    J[3][i] = z[0]
    J[4][i] = z[1]
    J[5][i] = z[2]

print(J) #complete manipulator jacobian

In [ ]:
print(on0) #end effector position

In [ ]:
qdot = list(map(float,input("qdot values: ").strip().split())) # rate of change of joint variables

In [ ]:
xdot = J@qdot
print("linear velocity: ", xdot[0], xdot[1], xdot[2],"\n")
print("angular velocity: ", xdot[3], xdot[4], xdot[5])